In [ ]:
import pandas as pd
import numpy as np
import os
import time
import datetime
from openai import OpenAI
import seaborn as sns
import pymssql
import re
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from threading import Thread
import functools
from tableone import TableOne
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu as mwu
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.statistics import multivariate_logrank_test
from sksurv.nonparametric import kaplan_meier_estimator
from sklearn.metrics import roc_curve, auc
import random
from threading import Thread
import functools
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve, auc
from lifelines.utils import concordance_index

conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
os.environ["OPENAI_API_KEY"] = ""  # enter openai api key

# simulated data generation

In [ ]:
def find_lognormal_parameters(b, c, percentile_indecimal):
    # Calculate z-scores for 1% and 99%
    z_percentile = norm.ppf(percentile_indecimal)


    # The median of the lognormal is the exp of the mean (mu) of the underlying normal
    mu = np.log(b)

    # Use the equations derived from lognormal properties to solve for sigma
    # a = exp(mu + z_1 * sigma) and c = exp(mu + z_99 * sigma)
    sigma = (np.log(c) - mu) / z_percentile
    a = np.exp(mu + norm.ppf(0.001) * sigma)
    c = np.exp(mu + norm.ppf(0.999) * sigma)

    return mu, sigma, a, c

In [ ]:
age_median=50
age_99p=90
height_men_median=175
height_men_99p=187
height_women_median=163
height_women_99p=175

In [ ]:
#men
targetsize=10000
continuous_data_list=['age','height_men']
df_synthetic=pd.DataFrame()
for i in range(len(continuous_data_list)):
    totalsize=100000
    if continuous_data_list[i]!='HDL':
        percentilenum=0.99
        percentilestr='99'
    else:
        percentilenum=0.01
        percentilestr='1'
        
    b=globals()[   (continuous_data_list[i]+'_median'  )  ]
    c=globals()[   (continuous_data_list[i]+'_'+percentilestr+  'p'  )  ]
    x1,x2,p1,p99=find_lognormal_parameters(b, c,percentilenum)
    
    s = np.random.lognormal(x1, x2, totalsize)
    s2=[]
    for j in s:
        if j<=p99 and j>=p1:
            s2.append(j)
    if len(s2)<targetsize:
        print('===========error==============',continuous_data_list[i])
    s2=s2[:targetsize]
    df_synthetic[continuous_data_list[i]]=s2

blacklist=[]

for i in range(targetsize//2):
    blacklist.append(0)
for i in range(targetsize//2,targetsize):
    blacklist.append(1)
     
random.shuffle(blacklist)  

df_synthetic['black']=blacklist
df_synthetic['sex']=1
df_synthetic.columns=['age','height','black','sex']
df_synthetic_men=df_synthetic.copy()
plt.hist(df_synthetic_men['height'],bins=40)
plt.show()

#women
targetsize=10000
continuous_data_list=['age','height_women']
df_synthetic=pd.DataFrame()
for i in range(len(continuous_data_list)):
    totalsize=100000
    if continuous_data_list[i]!='HDL':
        percentilenum=0.99
        percentilestr='99'
    else:
        percentilenum=0.01
        percentilestr='1'
        
    b=globals()[   (continuous_data_list[i]+'_median'  )  ]
    c=globals()[   (continuous_data_list[i]+'_'+percentilestr+  'p'  )  ]
    x1,x2,p1,p99=find_lognormal_parameters(b, c,percentilenum)
    
    s = np.random.lognormal(x1, x2, totalsize)
    s2=[]
    for j in s:
        if j<=p99 and j>=p1:
            s2.append(j)
    if len(s2)<targetsize:
        print('===========error==============',continuous_data_list[i])
    s2=s2[:targetsize]
    df_synthetic[continuous_data_list[i]]=s2

blacklist=[]

for i in range(targetsize//2):
    blacklist.append(0)
for i in range(targetsize//2,targetsize):
    blacklist.append(1)
     
random.shuffle(blacklist)  

df_synthetic['black']=blacklist
df_synthetic['sex']=0
df_synthetic.columns=['age','height','black','sex']
df_synthetic_women=df_synthetic.copy()
plt.hist(df_synthetic_women['height'],bins=40)
plt.show()

df_synthetic=pd.concat([df_synthetic_men,df_synthetic_women])
df_synthetic.reset_index(inplace=True,drop=True)

In [ ]:
df_synthetic

In [ ]:
rand_df=df_synthetic

txtlist=[]
for i in range(len(rand_df)):
    txt = ''
    txt += str(int(round(rand_df['age'][i],0)))
    txt += ' year old '
    
    if rand_df['black'][i]==1:
        txt += 'black '
    else:
        txt += 'white '
    
    if rand_df['sex'][i]==1:
        txt += 'male, '
    else:
        txt += 'female, '

    
    txt += 'height '
    txt += str(round(rand_df['height'][i],1))
    txt += ' cm'

    txtlist.append(txt)
    
rand_df['input_text']=txtlist

rand_df['age']= round(rand_df['age'], 0)
rand_df['height']= round(rand_df['height'], 1)

rand_df['eid']=list(rand_df.index)

In [ ]:
rand_df

In [ ]:
rand_df.to_csv('20241023_FVC_synthetic_20000.csv',index=False)

# GPT

In [ ]:
import pandas as pd
rand_df=pd.read_csv('20241023_FVC_synthetic_20000.csv')

In [ ]:
rand_df['input_text'][0]

In [ ]:
from openai import OpenAI
usercontent_global1='''Estimate Forced Vital Capacity (FVC) for the person below.

'''
usercontent_global2='''

Please answer exactly in the format below, without blank lines, and no further information or answer is required.
FVC=(in liters, round to two decimal places)'''

input_text=rand_df['input_text']
eid=rand_df['eid']

def timeout(timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function [%s] timeout [%s seconds] exceeded!' % (func.__name__, timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(timeout)
            except Exception as je:
                print ('error starting thread')
                raise je
            ret = res[0]
            #if isinstance(ret, BaseException):
            #    raise ret
            return ret
        return wrapper
    return deco

@timeout(0.1)
def ChatGPT_main(i,temper,responsenum):
    if input_text[i]!='':
        try:
            usercontent=usercontent_global1
            usercontent+=input_text[i]
            usercontent+=usercontent_global2

            messages=[
                    #{"role": "system", "content": usercontent_global3},
                    {"role": "user", "content": usercontent}
                ]

            client = OpenAI()
            completion=client.chat.completions.create(
                model='gpt-4',
                n=responsenum,
                temperature=temper,
                messages=messages
            )
            
            
            aa2=completion.choices
            for j in range(len(aa2)):
                aa=aa2[j].message.content.split('\n')
            #print(aa)
                remove_set={''}
                aaa=[ii for ii in aa if ii not in remove_set]

                temp1='blank'
                temp2=aaa[0].split('=')[1]
                temp3='blank'
                temp4='blank'
                temp5='blank'
                
               conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
                with conn:
                    with conn.cursor() as cur:
                        cur.execute(sql, (str(eid[i]),temp1,temp2,temp3,temp4,temp5))
                        conn.commit()

        except:
            temp1=np.nan
            temp2=np.nan
            temp3=np.nan
            temp4=np.nan
            temp5=np.nan
            
    else:
        temp1=np.nan
        temp2=np.nan
        temp3=np.nan
        temp4=np.nan
        temp5=np.nan
        


In [ ]:
i=4
temper=0.0
time1=datetime.datetime.now()
usercontent=usercontent_global1
usercontent+=input_text[i]
usercontent+=usercontent_global2

messages=[
        #{"role": "system", "content": usercontent_global3},
        {"role": "user", "content": usercontent}
    ]

client = OpenAI()
completion=client.chat.completions.create(
    model='gpt-4',
    n=1,
    temperature=temper,
    messages=messages
)

aa2=completion.choices
time2=datetime.datetime.now()
print(time2-time1)
#data_final_groupmean[data_final_groupmean['eid']==i]

In [ ]:
responsenumnumnum=1
for experimentnum in range(1,2):
    print('=========================================================')
    print('experimentnum',experimentnum)
    input_text=rand_df['input_text']
    ############################
    table_name="20241023_FVC1"
    iterations=1
    upto=20000
    ############################
    for temper in [0.0]:
        time1=datetime.datetime.now()
        while True:
            toggle=0
            for i in range(iterations):   
                newtablaname=table_name + '_' + str(int(experimentnum)) + '_' + str(i)
                try:
                    sql_createtable="CREATE TABLE [" + newtablaname +"""] 
                    (
                        eid    NVARCHAR(20),
                        system    NVARCHAR(max),
                        score      NVARCHAR(max),
                        category     NVARCHAR(max) ,
                        framingham  NVARCHAR(20),
                        ACC_AHA  NVARCHAR(20)
                    )

                    """
                    conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
                    with conn:
                        with conn.cursor() as cur:
                            cur.execute(sql_createtable)
                            conn.commit()
                    time.sleep(1)  
                except:
                    pass

                sql_statement="select * from ["+ newtablaname + "]"
                conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
                datasql = pd.read_sql(sql=sql_statement, con=conn)
                datasql=datasql.astype({'eid':int})
                cnt_df=datasql.groupby('eid')['score'].count()
                tempcount=0
                for j in range(upto):
                    try:
                        cnt=cnt_df[j]
                        if cnt<responsenumnumnum:
                            addcnt=int(responsenumnumnum-cnt)
                            sql = "INSERT INTO [" + newtablaname+"] (eid, system,score,category,framingham,ACC_AHA) VALUES (%s, %s, %s, %s, %s, %s)"
                            ChatGPT_main(j,temper,addcnt)
                            tempcount+=1

                    except:
                        sql = "INSERT INTO [" + newtablaname+"] (eid, system,score,category,framingham,ACC_AHA) VALUES (%s, %s, %s, %s, %s, %s)"
                        ChatGPT_main(j,temper,responsenumnumnum)
                        tempcount+=1
                    if j%100==0:
                        print(j)

                if tempcount==0:
                    toggle=1
            if toggle==1:
                break
            time.sleep(20)

        time.sleep(20)


        time2=datetime.datetime.now()
        print(time2-time1)
        #print('except_else_num = ',except_else_num)

# UKB data

In [ ]:
gpt_cindex=[]
validnum=[]
for experimentnum in range(1,2):
    print('=========================================================')
    print('experimentnum',experimentnum)
    rand_df=pd.read_csv('20241023_FVC_synthetic_20000.csv')
    rand_df['eid']=range(20000)
    input_text=rand_df['input_text']
    ############################
    table_name="20241023_FVC1"
    iterations=1
    upto=20000
    ############################
    resultsdf=pd.DataFrame(columns=['validnum','age', 'HDL', 'LDL', 'TG', 'HbA1c', 'Cr', 'Urate', 'CRP', 'SBP', 'DBP', 'BMI', 'WHratio', 'sex', 'HTN', 'DM', 'Dyslipidemia', 'Afib', 'CKD', 'CVDfamily', 'smoking','avg_gpt_score','c-index gpt','c-index acc/aha','c-index framingham'])


    for temper in [0.0]:
        templist=[]
        print('=========temperature========',temper)
        for i in range(iterations):
            newtablename=table_name + '_' + str(int(experimentnum)) + '_' + str(i)

            sql_statement="select * from ["+ newtablename + "]"
            conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
            globals()['data{}'.format(i)] = pd.read_sql(sql=sql_statement, con=conn)


        for j in range(iterations):
            dat=globals()['data{}'.format(j)]

            eid=dat['eid']
            score_gpt=dat['score']
            category=dat['category']
            score_framingham=dat['framingham']
            score_acc_aha=dat['ACC_AHA']

            score_gpt2=[]

            for i in range(len(score_gpt)):
                try:
                    score_gpt2.append(float(re.findall("\d+[.]\d+[%]",score_gpt[i])[0].split('%')[0]))
                except:
                    try:
                        score_gpt2.append(float(re.findall("\d+[%]",score_gpt[i])[0].split('%')[0]))
                    except:
                        try:
                            score_gpt2.append(float(re.findall("\d+[.]\d+",score_gpt[i])[0]))
                        except:
                            try:
                                score_gpt2.append(float(re.findall("\d+",score_gpt[i])[-1]))
                            except:
                                score_gpt2.append(np.nan)
                                #score_gpt2.append(float(score_framingham[i]))
                                #print(score_gpt[i],'====')


        data_final=pd.DataFrame({'eid':data0['eid'],'score':score_gpt2})
        data_final['eid']=data_final['eid'].astype(int)
        print(len(data_final))
        data_final=data_final.dropna()
        data_final.reset_index(inplace=True,drop=True)


In [ ]:
#data_final = data_final.groupby('eid')['score'].mean().reset_index()
df_merged=pd.merge(data_final,rand_df,how='inner',on='eid')

In [ ]:
columnslisttemp=list(df_merged.columns)
columnslisttemp=columnslisttemp[2:]
columnslisttemp=columnslisttemp[:-1]


#print(columnslisttemp)
#X=df_merged[['age','sex','HDL','LDL','TG','Chol','SBP','DBP','BMI','HTN','DM','smoking']]
X=df_merged[columnslisttemp]
y=df_merged['score']

line_fitter = LinearRegression()
line_fitter.fit(X, y)
print(columnslisttemp)
print(line_fitter.coef_)


In [ ]:
X2=X.copy()
X2['y']=y
X2.to_csv('20241023_FVC_R.csv',index=False)

In [ ]:
columnslisttemp=list(df_merged.columns)
columnslisttemp=columnslisttemp[2:]
columnslisttemp=columnslisttemp[:-1]


#print(columnslisttemp)
#X=df_merged[['age','sex','HDL','LDL','TG','Chol','SBP','DBP','BMI','HTN','DM','smoking']]
df_merged_temp=df_merged[df_merged['sex']==1]

X=df_merged_temp[columnslisttemp]
X.drop(['sex'],inplace=True,axis=1)
y=df_merged_temp['score']

line_fitter = LinearRegression()
line_fitter.fit(X, y)
print(columnslisttemp)
print(line_fitter.coef_)


In [ ]:
X2=X.copy()
X2['y']=y
X2.to_csv('20241023_FVC_R_male.csv',index=False)

In [ ]:
columnslisttemp=list(df_merged.columns)
columnslisttemp=columnslisttemp[2:]
columnslisttemp=columnslisttemp[:-1]


#print(columnslisttemp)
#X=df_merged[['age','sex','HDL','LDL','TG','Chol','SBP','DBP','BMI','HTN','DM','smoking']]
df_merged_temp=df_merged[df_merged['sex']==0]

X=df_merged_temp[columnslisttemp]
X.drop(['sex'],inplace=True,axis=1)
y=df_merged_temp['score']

line_fitter = LinearRegression()
line_fitter.fit(X, y)
print(columnslisttemp)
print(line_fitter.coef_)

In [ ]:
X2=X.copy()
X2['y']=y
X2.to_csv('20241023_FVC_R_female.csv',index=False)